In [ ]:
!pip install recommenders
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.model_selection import train_test_split
import pickle
from recommenders.evaluation.python_evaluation import ndcg_at_k

In [ ]:
train_data = pd.read_csv("train_20.csv")
test_data = pd.read_csv("test_20.csv")
ratings = pd.read_csv("ratings.csv")

In [ ]:
movie_index_map = pickle.load(open("movie_index_map.pkl", "rb"))
movie_vec_mat = pickle.load(open("movie_vector_matrix.pkl", "rb"))

In [ ]:
no_movies = len(ratings["movieId"].unique())
no_users = len(ratings["userId"].unique())

In [ ]:
# user df layers
user_input = tf.keras.Input(shape=(1,))
u_embedding_layer = tf.keras.layers.Embedding(no_users+1, 100, input_length=1)(user_input)
u_embedding_layer = tf.keras.layers.Flatten()(u_embedding_layer)
user_dense1 = tf.keras.layers.Dense(512, activation="relu")(u_embedding_layer)
drop4 = tf.keras.layers.Dropout(0.2)(user_dense1)
user_dense2 = tf.keras.layers.Dense(256, activation="relu")(drop4)

In [ ]:
# item df layers
item_input = tf.keras.Input(shape=(1, ))
i_embedding_layer = tf.keras.layers.Embedding(no_movies+1, 100, input_length=1)(item_input)
i_embedding_layer = tf.keras.layers.Flatten()(i_embedding_layer)
item_dense1 = tf.keras.layers.Dense(512, activation="relu")(i_embedding_layer)
drop3 = tf.keras.layers.Dropout(0.2)(item_dense1)
item_dense2 = tf.keras.layers.Dense(256, activation="relu")(drop3)

In [ ]:
#movie vector layer
movie_input = tf.keras.Input(shape=(100,))
#concatenation layer
concat = tf.keras.layers.concatenate([user_dense2, item_dense2, movie_input], axis=1)
concat = tf.keras.layers.concatenate([concat, movie_input])

# interaction network
inter_dense1 = tf.keras.layers.Dense(256, activation="relu")(concat)
inter_dense2 = tf.keras.layers.Dense(128, activation="relu")(inter_dense1)
inter_dense3 = tf.keras.layers.Dense(64, activation="relu")(inter_dense2)
output_layer = tf.keras.layers.Dense(1, activation="linear")(inter_dense3)

model = tf.keras.Model(inputs=[user_input, item_input, movie_input], outputs=output_layer)
model.compile(optimizer="adam", loss='mse')

In [ ]:
# training data

train_users, train_movies, train_labels, train_movieids = train["user_index"].values, train["movie_index"].values, train["rating"].values, train["movie"].values
train_movie_vecs = [movie_vec_mat[movie_index_map[movie_id]-1] for movie_id in train_movieids]

In [ ]:
model.fit([np.array(train_users), np.array(train_movies), np.array(train_movie_vecs)], np.array(train_labels), epochs=10, shuffle=True, batch_size=128, validation_split=0.2)

In [ ]:
m = tf.keras.metrics.RootMeanSquaredError()
test_users, test_movies, test_ratings, test_movieids = test_data["user_index"].values, test_data["movie_index"].values, test_data["rating"].values, test_data["movie"].values
test_movie_vecs = [movie_vec_mat[movie_index_map[mov]] for mov in test_movieids]
pred_ratings = model.predict([test_users, test_movies, np.array(test_movie_vecs)])
pred_ratings = np.array([rating[0] for rating in pred_ratings])
test_data["predictions"] = pred_ratings
mae = tf.keras.metrics.mean_absolute_error(np.array(test_data["rating"]), pred_ratings)
print(f"Mean Absolute Error: {np.mean(mae)}")
m.update_state(pred_ratings, test_ratings)
print(f"Root Mean Square Error: {m.result().numpy()}")

In [ ]:
mean_precision = 0
mean_recall = 0
for user in train_data["user_index"].unique():
  test_set = test_data[test_data["user_index"] == user].reset_index()
  ratings_list = list(zip(test_set["rating"], test_set["predictions"]))
  ratings_list.sort(key=lambda x: x[1], reverse=True)

  total_relevant = sum(rat >= 4.0 for rat, pred in ratings_list)
  total_recommended = sum(pred >= 4.0 for rat, pred in ratings_list[:10])
  total_rec_rel = sum(rat >= 4.0 and pred >= 4.0 for rat, pred in ratings_list[:10])

  mean_precision += (total_rec_rel/total_recommended) if total_recommended > 0 else 0
  mean_recall += (total_rec_rel/total_relevant) if total_relevant > 0 else 0
  
print(f"Precision: {mean_precision/len(train_data['user_index'].unique())}")
print(f"Recall: {mean_recall/len(train_data['user_index'].unique())}")

In [ ]:
top_n_test_data = test_data.groupby("user", as_index=False).apply(lambda x: x.nlargest(10, "rating")).reset_index(drop=True)

In [ ]:
ndcg_at_k(top_n_test_data, test_data, col_user="user", col_item="movie", col_rating="rating", col_prediction="predictions")